# Annotations

In [1]:
from ipynb.fs.full.koselleck import *

In [2]:
dfranks=get_all_word_rankings()
dfranks

,+Abs,+Conc,+Active,+Passive,+Ambig,-Ambig,+Collective,+Indiv,+Freq,-Freq,...,+Sing,+Plural,+Strong,+Weak,+Time,+Space,+Virtue,+Vice,+Woman,+Man
word,,,,,,,,,,,,,,,,,,,,,
lack,1,9976,9906,71,7336,2640,665,9312,885,9092,...,NaN,NaN,539,9438,6683,3294,4,9973,9252,725
lax,2,9975,6988,2989,5894,4083,6059,3918,7472,2505,...,NaN,NaN,744,9233,110,9867,269,9708,9623,354
tact,3,9974,9735,242,9245,732,5945,4032,1431,8546,...,NaN,NaN,10,9967,4113,5864,2,9975,8317,1660
lucid,4,9973,1460,8517,3606,6371,2533,7444,4006,5971,...,NaN,NaN,14,9963,82,9895,114,9863,9876,101
beset,5,9972,9343,634,5996,3981,549,9428,3259,6718,...,NaN,NaN,3092,6885,1966,8011,1486,8491,8613,1364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
conduit,9972,5,619,9358,6519,3458,686,9291,9256,721,...,NaN,NaN,9392,585,9895,82,9539,438,133,9844
hewn,9973,4,121,9856,6999,2978,8940,1037,9589,388,...,NaN,NaN,9939,38,9620,357,9974,3,1613,8364
slate,9974,3,7,9970,9789,188,9293,684,9925,52,...,1916.0,45.0,9397,580,9943,34,9907,70,6412,3565


In [3]:
ANNOCOLS = ['Changes','From','To','Notes','is_valid']

In [5]:
def start_sheet_changes(min_peaks=1):
    dfchangepoints=get_signif_novelty_scores(
        min_peaks=min_peaks
    ).sort_values(
        'foote_novelty_z',ascending=False
    ).drop_duplicates(
        'word',keep='first'
    ).set_index('word').join(
        get_all_novelty_scores().groupby('word').mean(),
        rsuffix='_avg_sign'
    ).join(
        get_all_novelty_scores().groupby('word').mean(),
        rsuffix='_avg'
    ).join(
        get_all_word_rankings()
    ).sort_values(
        'foote_novelty_z_avg_sign',
        ascending=False
    ).rename({'period':'changepoint'},axis=1)
    dfchangepoints['rank'] = dfchangepoints.foote_novelty_z_avg_sign.rank(
        ascending=False
    ).apply(int)
    
    for ac in ANNOCOLS: dfchangepoints[ac]=''
    prefcols=['rank','changepoint'] + ANNOCOLS + ['word_num_peaks','foote_novelty_z_avg_sign']
    dfchangepoints = dfchangepoints[
        prefcols + sorted([c for c in dfchangepoints.columns if c not in set(prefcols)])
    ]
    return round(dfchangepoints,2)

,neighb1_len,neighb2_len,metaneighb_len,lnm,lnm_z,lnm_perc,rank_word,lnm_perc_word
word,,,,,,,,
greed,25.0,25.0,50.000000,1.417704,10.455362,99.985492,1.0,100.000000
confiding,25.0,25.0,50.000000,1.246019,9.088643,99.949366,2.0,99.991128
lack,25.0,25.0,50.000000,1.218931,8.873013,99.939847,3.0,99.982257
fee,25.0,25.0,50.000000,1.204533,8.758393,99.933482,4.0,99.973385
muff,25.0,25.0,50.000000,1.187863,8.625689,99.924250,5.0,99.964514
...,...,...,...,...,...,...,...,...
pigs,25.0,25.0,33.666667,0.009299,-0.756375,0.200183,11268.0,0.044358
fowls,25.0,25.0,35.833333,0.008921,-0.759381,0.205574,11269.0,0.035486
oxen,25.0,25.0,32.416667,0.007861,-0.767819,0.145479,11270.0,0.026615


In [4]:
# odf=get_changeranks_sattelzeit().join(start_sheet_changes())
# odf

In [5]:
# read_sheet()

In [20]:
def update_sheet_changes(df=None):
    df=start_sheet_changes() if df is None else df
    df_now=read_sheet()
    if len(df_now):
        for col in ANNOCOLS:
            df[col]=df_now[col]
    df=df.fillna('')
    prefcols=['rank_word']
    sufcols=['lnm_perc_word','lnm_perc','lnm_z','lnm']
    pref=prefcols + ANNOCOLS + sufcols
    df=df[pref + [c for c in df.columns if c not in set(pref)]]
    df.rank_word=df.rank_word.apply(int)
    df=round(df,2)
    df=df.sort_values('rank_word')
    write_sheet(df)
    return df

In [6]:
update_sheet_changes()

## Gather data from anno

In [7]:
dfanno=read_sheet()

In [16]:
DFANNO=None
def get_dfanno(force=False,dbkey='dfanno'):
    global DFANNO
    if not force:
        if DFANNO is not None: return DFANNO
        with get_veclib('anno') as vl:
            if dbkey in vl: return vl[dbkey]
    print('Reading annotations from online')
    DFANNO=df=read_sheet()
    with get_veclib('anno') as vl:
        vl[dbkey]=df
        vl.commit()
    return DFANNO

In [18]:
dfanno=get_dfanno(force=False)
dfanno

,rank_word,Changes,From,To,Notes,is_valid,lnm_perc_word,lnm_perc,lnm_z,lnm,...,is_signif_avg,is_signif_avg_sign,p_peak,p_peak_avg,p_peak_avg_sign,p_trough,p_trough_avg,p_trough_avg_sign,period_avg,period_avg_sign
word,,,,,,,,,,,,,,,,,,,,,
greed,1,,,,,n,100,99.99,10.46,1.42,...,,,,,,,,,,
maladies,7018,,,,,,37.75,36.97,-0.4,0.05,...,,,,,,,,,,
barking,7019,,,,,,37.74,36.36,-0.4,0.05,...,,,,,,,,,,
dwarf,7020,,,,,,37.73,36.64,-0.4,0.05,...,,,,,,,,,,
uncouth,7021,,,,,,37.72,36.64,-0.4,0.05,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rotation,7900,,,,,,29.92,29.47,-0.46,0.05,...,0.04,0.04,0.04,0.92,0.92,1,0.95,0.95,1806.2,1806.2
lamentable,2444,,,,,,78.33,74.3,0.07,0.11,...,0.05,0.05,0.03,0.91,0.91,1,0.71,0.71,1822.5,1822.5
bullion,6764,,,,,,40,38.18,-0.39,0.06,...,0.05,0.05,0.02,0.81,0.81,1,0.9,0.9,1792.5,1792.5


In [20]:
def get_words_anno_valid():
    dfanno=get_dfanno()
    return dfanno[dfanno.is_valid.str.startswith('y')].index
def get_words_anno_invalid():
    dfanno=get_dfanno()
    return dfanno[dfanno.is_valid.str.startswith('n')].index
        

In [24]:
len(get_words_anno_valid())

77

In [42]:
def _respell_change(x):
    x=x.replace('z','s')
    x
    return x

def get_changes_anno(min_count=3):
    odf=pd.DataFrame([
        {'word':w, 'Change':cx.strip().title(), **dict(row)}
        for w,row in get_dfanno().query('Changes!=""').iterrows()
        for cx in row.Changes.split(',')
    ])
    odf.Change=odf.Change.apply(_respell_change)
    if min_count: odf=odf.groupby('Change').filter(lambda x: len(x)>=min_count)
    return odf

In [46]:
# get_changes_anno()